In [25]:
import torch.nn as nn
from torch.autograd import Variable as V
import torch as th
from torchvision import models
import os
import torch.optim as optim
import random
import numpy as np
import cv2 as cv2
from alexlstm import AlexLSTM
from datasetutil import DatasetUtil
from importlib import reload

%load_ext autoreload
%autoreload 2

batch_size = 5
time_stamp = 20
frame_offset_per_time_stamp = 10
train_dataset = os.listdir("img/")
total_img_num = len(train_dataset)
iteration_per_epoch = int(total_img_num / (batch_size*time_stamp))

def train():
    net = AlexLSTM()
    util = DatasetUtil()
    criterion = nn.MSELoss(False)
    lr = 0.0001
    min_loss = 100
    for epoch in range(20):  # loop over the dataset multiple times
        running_loss = 0.0
        for i in range(iteration_per_epoch):
            x,y = util.fetch_image_and_label(batch_size, time_stamp, frame_offset_per_time_stamp, total_img_num)
            
            # wrap them in Variable
            x = V(th.from_numpy(x).float())
            y = V(th.from_numpy(y).float())

            optimizer = optim.Adam(net.parameters(), lr=lr)
            optimizer.zero_grad()# zero the parameter gradients
            # forward + backward + optimize
            predict = net(x)

            print("------------ PREDICT ------------")
            print(predict)
            print("------------ LABEL --------------")
            print(y)
            loss = criterion(predict, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.data[0]
            if running_loss <= min_loss :
                min_loss = running_loss
                print("--- Found smaller loss ---")
                th.save(net.state_dict(), 'weight/%d_%s.p' % (i, epoch))
            print('[epoch : %d, iteration : %5d] loss: %.3f' % (epoch, i, running_loss))
            running_loss = 0.0
        print("Saving model per epoch...")
        th.save(net.state_dict(), 'weight/epoch_%s.p' % (epoch))
    print('Finished Training')

train()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
------------ PREDICT ------------
Variable containing:

Columns 0 to 9 
 0.1202  0.1258  0.1386  0.1226  0.1267  0.1230  0.1293  0.1308  0.1282  0.1237
 0.1244  0.1319  0.1155  0.1200  0.1164  0.1105  0.1197  0.1084  0.1257  0.1191
 0.1242  0.1220  0.1190  0.1090  0.1230  0.1259  0.1330  0.1378  0.1240  0.1464
 0.1258  0.1226  0.1195  0.1108  0.1089  0.1077  0.1054  0.1143  0.1130  0.1072
 0.1252  0.1241  0.1294  0.1207  0.1242  0.1200  0.1172  0.1254  0.1274  0.1220

Columns 10 to 19 
 0.1249  0.1220  0.1164  0.1285  0.1281  0.1179  0.1321  0.1300  0.1151  0.1340
 0.1232  0.1194  0.1168  0.1158  0.1033  0.1173  0.1214  0.1122  0.1043  0.1065
 0.1491  0.1375  0.1370  0.1357  0.1256  0.1318  0.1319  0.1198  0.1155  0.1152
 0.1039  0.1097  0.1131  0.1055  0.1148  0.1178  0.1138  0.1168  0.1129  0.1190
 0.1227  0.1291  0.1231  0.1222  0.1190  0.1336  0.1244  0.1358  0.1259  0.1213
[torch.FloatTensor of

[epoch : 0, iteration :     4] loss: 28609.480
------------ PREDICT ------------
Variable containing:

Columns 0 to 9 
 0.1364  0.1576  0.2023  0.2120  0.2278  0.2375  0.2168  0.2551  0.2268  0.2447
 0.1369  0.1581  0.1915  0.2067  0.2230  0.2070  0.2248  0.2131  0.2037  0.2005
 0.1433  0.1712  0.1876  0.1864  0.2184  0.2246  0.2258  0.2151  0.2363  0.2316
 0.1465  0.1544  0.1960  0.2114  0.2099  0.2401  0.2330  0.2484  0.2478  0.2433
 0.1497  0.1634  0.2000  0.2018  0.1943  0.2185  0.2326  0.2508  0.2388  0.2392

Columns 10 to 19 
 0.2252  0.2311  0.2305  0.2306  0.2673  0.2230  0.2009  0.2052  0.2382  0.2134
 0.2095  0.2333  0.2251  0.2324  0.2185  0.2346  0.2179  0.2397  0.2451  0.2467
 0.2434  0.2219  0.2376  0.2388  0.2182  0.2203  0.2528  0.2467  0.2498  0.2343
 0.2420  0.2461  0.2349  0.2305  0.2672  0.2541  0.2550  0.2289  0.2434  0.2534
 0.2577  0.2695  0.2424  0.2651  0.2710  0.2534  0.2771  0.2440  0.2377  0.2498
[torch.FloatTensor of size 5x20]

------------ LABEL   -------

[epoch : 0, iteration :     9] loss: 18000.207
------------ PREDICT ------------
Variable containing:

Columns 0 to 9 
 0.1754  0.2462  0.3254  0.3654  0.4305  0.4750  0.4334  0.4848  0.5258  0.4436
 0.1802  0.2580  0.2909  0.3617  0.4262  0.4023  0.4778  0.4795  0.5294  0.4975
 0.1929  0.2700  0.3139  0.3797  0.3755  0.4583  0.5535  0.5557  0.6082  0.6109
 0.1648  0.2587  0.2633  0.3291  0.4438  0.4199  0.4178  0.4992  0.4305  0.4204
 0.1667  0.2364  0.3100  0.3779  0.4177  0.4545  0.4601  0.5383  0.5321  0.4456

Columns 10 to 19 
 0.5238  0.5132  0.5530  0.5397  0.5196  0.5439  0.5007  0.3966  0.5481  0.5586
 0.5663  0.5716  0.5468  0.4863  0.5568  0.5089  0.5396  0.5185  0.5844  0.5421
 0.5849  0.4788  0.5751  0.5991  0.5877  0.5908  0.5224  0.5896  0.6050  0.6316
 0.5881  0.5203  0.5948  0.4953  0.5553  0.5161  0.6028  0.5350  0.4772  0.5720
 0.4888  0.5614  0.6278  0.5326  0.5060  0.5791  0.5653  0.5027  0.5485  0.5694
[torch.FloatTensor of size 5x20]

------------ LABEL   -------

KeyboardInterrupt: 